In [34]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import os
import random
import itertools
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
!pip install efficientnet_pytorch
import efficientnet_pytorch

In [35]:
class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b4')
        self.base_model._fc = nn.Linear(in_features=1792, out_features=1, bias=True)
        
    def forward(self, image):
        out = self.base_model(image)
        out = torch.sigmoid(out)
        return out

In [36]:
model = EfficientNet()
model.load_state_dict(torch.load('../input/classifier/model_classification.pth', map_location=torch.device('cpu')))
model.eval()

In [37]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [38]:
path = '../input/melanoma-classification/Melanoma New/test'
imgs = os.listdir(path)
imgs.sort()
len(imgs)

In [41]:
submission = pd.DataFrame({'image_name':imgs, 'target':[0]*len(imgs)})
submission

In [44]:
targets = []
for i in imgs:
    path_img = path + '/' + i
    img = Image.open(path_img)
    transformed_img = transform(img)
    transformed_img = torch.unsqueeze(transformed_img, 0)
    y_pred = model(transformed_img)
    targets.append(y_pred.item())

In [49]:
names = []
for i in imgs:
    names.append(i[:-4])
submission['image_name'] = names

In [50]:
submission['target'] = targets
submission

In [51]:
submission.to_csv('submission.csv', index = False)